<a href="https://colab.research.google.com/github/pmargarete/CCADMACL_EXERCISES_COM222/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [81]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

In [82]:
#FILTER Variance Threshold
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.2)
sel = selector.fit(df)
sel_index = sel.get_support()
df_vt = df.iloc[:, sel_index]

In [83]:
df_vt

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [87]:
#WRAPPER

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

threshold = 5 #number of most relevant features
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(model_rf, n_features_to_select=5, step=1)

selector = selector.fit(df.drop('AveRooms', axis=1), df['AveRooms'])
selector_ind = selector.get_support()

df_rfe = df.iloc[:, sel_index]

df_rfe = df.drop('AveRooms', axis=1).iloc[:, selector_ind]

df_rfe['AveRooms'] = df['AveRooms']

df_rfe.columns

Index(['MedInc', 'HouseAge', 'AveBedrms', 'Population', 'MedHouseVal',
       'AveRooms'],
      dtype='object')

In [90]:
# EMBEDDED

from sklearn.feature_selection import SelectFromModel

model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_rf.fit(df.drop('AveRooms', axis=1), df['AveRooms'])

sel_sfm = SelectFromModel(model_rf, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_sfm = df.drop('AveRooms', axis=1).iloc[:, sel_sfm_index]
df_sfm['AveRooms'] = df['AveRooms']
df_sfm.columns

Index(['AveBedrms', 'AveRooms'], dtype='object')

In [91]:
from sklearn.model_selection import train_test_split

# with selected features and target
X = df_vt  # Features
y = df_sfm #['AveRooms']  #Target


In [93]:
from sklearn.datasets import fetch_california_housing
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#X_train_embedded, X_test_embedded, y_train, y_test = train_test_split(X_embedded, y, test_size=0.2, random_state=42)

#X_filter = df_vt[['MedInc',	'HouseAge',	'AveRooms',	'AveBedrms',	'Population',	'AveOccup',	'Latitude',	'Longitude',	'MedHouseVal']]  # Replace with selected features after Filter method
#X_train_filter, X_test_filter, _, _ = train_test_split(X_filter, y, test_size=0.2, random_state=42)

#X_wrapper = df_rfe[['MedInc', 'HouseAge', 'AveBedrms', 'Population', 'MedHouseVal', 'AveRooms']]  # Replace with selected features after Wrapper method
#X_train_wrapper, X_test_wrapper, _, _ = train_test_split(X_wrapper, y, test_size=0.2, random_state=42)

def calculate_rmse(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [95]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

rmse = calculate_rmse(model, X_train, X_test, y_train, y_test)
#rmse_filter = calculate_rmse(model, X_train_filter, X_test_filter, y_train, y_test)
#rmse_wrapper = calculate_rmse(model, X_train_wrapper, X_test_wrapper, y_train, y_test)
#rmse_embedded = calculate_rmse(model, X_train_embedded, X_test_embedded, y_train, y_test)

print(f"RMSE: {rmse}")
#print(f"Wrapper RMSE: {rmse_wrapper}")
#print(f"Embedded RMSE: {rmse_embedded}")

RMSE: 0.10724892636856957


In [75]:
#RootMeanSquaredError
#Filter
#Wrapper
#Embedded